## ネットから賃貸情報を読み込む

## Googleドライブをマウントする

In [ ]:
from google.colab import drive
googledrive = '/content/drive'                                   # GoogleColaboratory の マウントポイント
drive.mount( googledrive )                                        # マウントポイントにGoogleDriveをマウントする
mydrive = googledrive + '/MyDrive'                           # 上でマウントしたGoogleDriveのMyDriveの絶対path
pylearn_dir = mydrive + "/Colab Notebooks/pylearn" # 空白がある場合は、ダブルクォーテーションで囲う
html_dir = mydrive + '/html'                                      # /content/drive/MyDrive/html HTML文書の置き場
suumo_dir = html_dir + '/suumo'                               # スーモ賃貸情報HTML文書の置き場
input_html_dir = 'file://'+ suumo_dir

# GoogleBotのユーザーエージェント（User Agent）
USER_AGENT="Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"

## wget コマンドで賃貸情報ページを取得する

In [ ]:
import datetime

t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
today = now.strftime('%Y-%m-%d')
print(today)

In [ ]:
# 足立区新着順50件
url_base = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13121&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=09&pc=50"

for i in range(134): # 全134ページ
    if i > 0:
        page='&page=' +  str(i+1)
    else:
        page=''
    outFile = 'adachi_50_' + today + '_' + str( i + 1).zfill(3) + '.html'
    url = url_base + page
    ! wget -U "$USER_AGENT" -O "$suumo_dir/$outFile" "$url"

## Playwrightをインストールして、Pythonでimportする

In [ ]:
! pip install playwright
! playwright install-deps
! playwright install webkit
import nest_asyncio
nest_asyncio.apply()
import asyncio
from playwright.async_api import async_playwright
! pip install --upgrade -q gspread

## Googleシートを読み込めるようにする

In [3]:
import gspread
from google.colab import auth
from google.colab import drive
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gss_key = '1FJB_uZsk89FVG9XddKwmOlIsmK_DHeCfwmLpdj0EUWM'
sheet = 'chintai'

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
workbook = gc.open_by_key(gss_key)
worksheet = workbook.worksheet(sheet)

## PlaywrightでHTMLを読み込んでGoogleシートに書き込む

In [ ]:
RESOURCE_EXCLUSTIONS = [ 'image',  'stylesheet',  'media',  'font',  'script' ]
base_url = 'https://suumo.jp'

html_files = ! ls -1 $suumo_dir/*.html

print(html_files)
async def main():
    async with async_playwright() as p:
        browser = await p.webkit.launch()
        context = await browser.new_context(user_agent=USER_AGENT)
        page = await context.new_page()
        await page.route("**/*", lambda route: route.abort()
            if route.request.resource_type in RESOURCE_EXCLUSTIONS
            else route.continue_()
        )
        for i in range(len(html_files)):
            print(html_files[ i ])

            await page.goto('file://' + html_files[ i ])
            whole_locator = page.locator('#js-bukkenList > ul > li > div')
            case_locator = whole_locator.locator("div.cassetteitem-detail")
            case_count = await case_locator.count()

            for j in range(case_count):

                  enum_locator = page.locator("#js-bukkenList > ul > li > div").nth( j )
                  label_locator = enum_locator.locator("div.cassetteitem-detail > div.cassetteitem-detail-body > div > div.cassetteitem_content-label > span")
                  title_locator = enum_locator.locator("div.cassetteitem-detail > div.cassetteitem-detail-body > div > div.cassetteitem_content-title")
                  address_locator = enum_locator.locator("div.cassetteitem-detail > div.cassetteitem-detail-body > div > div.cassetteitem_content-body > ul > li.cassetteitem_detail-col1")
                  station_locator = enum_locator.locator("div.cassetteitem-detail > div.cassetteitem-detail-body > div > div.cassetteitem_content-body > ul > li.cassetteitem_detail-col2")
                  age_locator = enum_locator.locator("div.cassetteitem-detail > div.cassetteitem-detail-body > div > div.cassetteitem_content-body > ul > li.cassetteitem_detail-col3 > div:nth-child(1)")
                  numoffloors = enum_locator.locator("div.cassetteitem-detail > div.cassetteitem-detail-body > div > div.cassetteitem_content-body > ul > li.cassetteitem_detail-col3 > div:nth-child(2)")

                  floor_locator = enum_locator.locator("div.cassetteitem-item > table > tbody:nth-child(2) > tr > td:nth-child(3)")
                  rent_locator = enum_locator.locator("div.cassetteitem-item > table > tbody:nth-child(2) > tr > td:nth-child(4) > ul > li:nth-child(1) > span > span")
                  fee_locator = enum_locator.locator("div.cassetteitem-item > table > tbody:nth-child(2) > tr > td:nth-child(4) > ul > li:nth-child(2) > span")
                  reward_locator = enum_locator.locator("div.cassetteitem-item > table > tbody:nth-child(2) > tr > td:nth-child(5) > ul > li:nth-child(1) > span")
                  layout_locator = enum_locator.locator("div.cassetteitem-item > table > tbody:nth-child(2) > tr > td:nth-child(6) > ul > li:nth-child(1) > span")
                  square_locator = enum_locator.locator("div.cassetteitem-item > table > tbody:nth-child(2) > tr > td:nth-child(6) > ul > li:nth-child(2) > span")
                  link_locator = enum_locator.locator("div.cassetteitem-item > table > tbody:nth-child(2) > tr > td.ui-text--midium.ui-text--bold > a")

                  label  = await label_locator.inner_text()
                  title = await title_locator.inner_text()
                  address = await address_locator.inner_text()
                  station = await station_locator.inner_text()
                  age = await age_locator.inner_text()
                  numoffloors = await numoffloors.inner_text()
                  floor = await floor_locator.inner_text()
                  rent = await rent_locator.inner_text()
                  fee = await fee_locator.inner_text() if fee_locator != None else ''
                  reward = await reward_locator.inner_text()
                  layout = await layout_locator.inner_text()
                  square = await square_locator.inner_text()
                  link = await link_locator.get_attribute('href')

                  sheet_range = 'B' + str( ( i * 50 )  + j + 2 ) + ':N' + str( ( i * 50 ) + j + 2 )
                  cell_list = worksheet.range( sheet_range )
                  cell_list[0].value = label
                  cell_list[1].value = title
                  cell_list[2].value = address
                  cell_list[3].value = station
                  cell_list[4].value = age
                  cell_list[5].value = numoffloors
                  cell_list[6].value = floor
                  cell_list[7].value = rent
                  cell_list[8].value = fee
                  cell_list[9].value = reward
                  cell_list[10].value = layout
                  cell_list[11].value = square
                  cell_list[12].value = base_url + link
                  worksheet.update_cells(cell_list)

        await browser.close()

asyncio.run(main())